# **AM 207**: Homework 5

Verena Kaynig-Fittkau and Pavlos Protopapas  <br>
**Due: 11.59 P.M. Thursday April 14th, 2016**

### Note: This homework is only for one week

### Instructions:

+ Upload your answers in an ipython notebook to Canvas.

+ We will provide you imports for your ipython notebook. Please do not import additional libraries.

+ Your individual submissions should use the following filenames: AM207_YOURNAME_HW5.ipynb

+ Your code should be in code cells as part of your ipython notebook. Do not use a different language (or format). 

+ **Do not just send your code. The homework solutions should be in a report style. Be sure to add comments to your code as well as markdown cells where you describe your approach and discuss your results. **

+ Please submit your notebook in an executed status, so that we can see all the results you computed. However, we will still run your code and all cells should reproduce the output when executed. 

+ If you have multiple files (e.g. you've added code files or images) create a tarball for all files in a single file and name it: AM207_YOURNAME_HW5.tar.gz or AM207_YOURNAME_HW5.zip


### Have Fun!
_ _ _ _ _

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")

import time
import timeit

import scipy.stats 
import pandas as pd
import pymc as pm

import re
import numpy as np

import string

# Problem 1: HMM... I Think Your Text Got Corrupted!

In this problem you should use a Hidden Markov Model to correct typos in a text without using a dictionary. Your data is in two different text files:

* `Shakespeare_correct.txt` contains the words of some sonnets from Shakespeare
* `Shakespeare_typos.txt` contains the same text, but now some of the characters are corrupted

For convenience both text files only contain lower case letters a-z and spaces. 

First build a first order HMM:
* What are the hidden states and what are the observed states?
* What should you do to generate your HMM probability matrices?
* For some of the HMM parameters, you won't have enough training data to get representative probabilities.  For example, some of your probabilites might be 0. You should address this problem by adding a small pseudocount, similar to the motif finding problem from a previous assignment. 
* Implement the Viterbi algorithm and run it on a test portion that contains errors. Show that your Viterbi implementation can improve text of length 100, 500, 1000, and 2000. Note: To do this correctly you would have to withhold the part of the text that you use for testing when you estimate the parameters for you HMM. For the sake of this homework it is ok though to report training error instead of test error. Just be aware that the correction rate you are reporting most likely is a very optimistic estimate. 
* What correction rate do you get?

**Important**: Wikipedia has a nice article on [Viterbi](https://en.wikipedia.org/wiki/Viterbi_algorithm). **Please do not use the python implementation from this article!** (The lecture notebook also has the version from Wikipedia). Using dictionaries for Viterbi is really not intuitive and using numpy is typically faster. The article has very nice pseudo code that should enable you to easily program Viterbi by yourself. Please also refrain for this problem from using any other third party implementations. 

Now for a second order HMM:
By using a second order HMM, you should be able to get a better correction rate. 
* Give an intuitive explanation why a second order HMM should give better results.
* Implement your second order text correction. Hint: If you think a bit about the model you won't even have to change your Viterbi implementation. 
* Compare your correction rates against the first order model for text length of 100 and 500, (you can do 1000 as well if your computer is fast enough). 
* How well would your implementation scale to HMMs of even higher order? 

### Hidden States: The true characters
### Observed States: The characters in Shakespeare_typos

# First Order:

### The HMM probability matrices are 27 by 27 matrices.  
### In the emission Matrix, each row represents the observable characters (26 letters plus space), and each column represents the true characters (same possible range as the observable characters).  
### The transition matrix has starting characters for each row, and following characters for each column.  To construct the transition matrix, I go through the correct text and count how many times each character follows each character.  This can then be normalized row-wise to get the probability of having each character follow the character that is represented by that row.  
### The transition matrix has hidden states (characters) as the rows, and observed states as the columns.  To construct the emission matrix, I go through both lines of text and count how many times each character is present in the typo text given the true character.  


In [6]:
s_correct

In [7]:
s_correct[0:10]

array(['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that',
       'thereby', 'beautys', 'rose'], 
      dtype='|S16')

In [9]:
# Put data into strings

with open('Shakespeare_correct.txt') as file:
    s_correct = file.read()
    
with open('Shakespeare_typos.txt') as file:
    s_typo = file.read()

In [218]:
# A dictionary to relate characters to indices
char_dict = {}
for i,char in enumerate(string.lowercase+' '):
    char_dict.update({char:i})

In [41]:
# Create emission matrix

emission = np.zeros((27,27))
for i in range(len(s_correct)):
    # Find characters
    char_correct = s_correct[i]
    char_typo = s_typo[i]
    
    # Add a count to the emission matrix at the corresponding position
    emission[char_dict[char_correct],char_dict[char_typo]] += 1
    
# Normalize the rows
for i in range(emission.shape[0]):
    row = emission[i,:]
    emission[i,:] = row/np.sum(row)

In [42]:
# Create transition matrix

transition = np.zeros((27,27))
for i in range(len(s_correct)-1):
    # Get the character and the one that follows it
    first_char = s_correct[i]
    next_char = s_correct[i+1]
    
    # Add a count to the transition matrix for each combination
    transition[char_dict[first_char],char_dict[next_char]] += 1
    
# Normalize the rows
for i in range(transition.shape[0]):
    row = transition[i,:]
    transition[i,:] = row/np.sum(row)

In [35]:
# Create starting probabilities.  This could be done a few ways, but what I 
#    implement below is the probability of each letter being the initial 
#    character in a word in the true text, since the first character in the
#    typo text is the beginning of a word.  

start_probs = np.zeros(27)

# Split the words
words = s_correct.split(' ')

for word in words:
    first_char = word[0]
    start_probs[char_dict[first_char]] += 1
    
# Normalize
start_probs = start_probs/np.sum(start_probs)

In [47]:
# I create another dictionary to match characters to indices:
index_dict = {}

for i,char in enumerate(string.lowercase+' '):
    index_dict.update({i:char})

In [278]:
def viterbi(obs, start_pr=start_probs, em=emission, tr=transition, \
            e_pseudo = 0.000001, t_pseudo = 0.000001, p_pseudo=0.000001):
    # add pseudocounts to make all elements non-zero
    e = em + e_pseudo
    t = tr + t_pseudo
    start_p = start_pr + p_pseudo
    
    # Array of probabilities for each character at each step in the document
    viterbi_probs = np.zeros((27,len(obs)))

    # Initial state:
    first_char = obs[0]
    # I use log probabilities to avoid underflow
    viterbi_probs[:,0] = np.log(start_p) + np.log(e[char_dict[first_char],:])

    for i in range(1,len(obs)):
        # At each step, find the character and use it to index the appropriate matrix entries
        char = obs[i]
        char_idx = char_dict[char]
        
        for j in range(len(char_dict)):
            # Find the max of the log probabilities for all possible characters
            prob = max([(viterbi_probs[k,i-1] + np.log(t[k,j]) + np.log(e[j,char_idx]))\
                       for k in range(len(char_dict))])
            viterbi_probs[j,i] = prob
            
    # Predictions are the max probability for each step
    char_predictions = np.argmax(viterbi_probs,axis=0)
    
    # Translate the max probabilities back to characters
    s_viterbi = ''
    for i in char_predictions:
        s_viterbi += index_dict[i]
        
    return s_viterbi

In [279]:
# I create a function to calculate the accuracy of a corrupted text:
def accuracy(corrupt, correct):
    # count of total errors
    errors = 0.
    for i in range(len(corrupt)):
        if corrupt[i] != correct[i]:
            errors += 1.
    return 1. - errors/len(correct)

## Overall accuracy of the typo text:

In [280]:
accuracy(s_typo,s_correct)

0.8374634600053149

#### Below I define a method to test the algorithm on a random slice of a specified length.  To get a better idea of how the algorithm performs in general, I will run many samples of a fixed length and return accuracies with errors.  

In [281]:
# I create a function to test the algorithm on random slices of a specified length:
def test_viterbi(length):
    # Random starting position
    start = np.random.randint(low=0, high = len(s_correct)-length)
    
    # Pick out the random segment:
    typo_segment = s_typo[start:start+length]
    correct_segment = s_correct[start:start+length]
    
    # Run viterbi:
    corrected_segment = viterbi(typo_segment)
    
    # Return accuracy of the original segment and the corrected segment
    return accuracy(typo_segment, correct_segment), accuracy(corrected_segment, correct_segment)

In [282]:
def test_sample(size,samples):
    print "For text of length {}, based on {} samples:".format(size,samples)
    samples = np.zeros((samples,2))
    for i in range(samples.shape[0]):
        orig, corrected = test_viterbi(size)
        samples[i,0] = orig
        samples[i,1] = corrected
    print "Original document accuracy: {:.3f} +/- {:.3f}".format(np.mean(samples[:,0]),np.std(samples[:,0]))
    print "Viterbi-corrected accuracy: {:.3f} +/- {:.3f}".format(np.mean(samples[:,1]),np.std(samples[:,1]))

## Results:

In [283]:
for i in [100, 500, 1000, 2000]:
    test_sample(i,30)
    print 

For text of length 100, based on 30 samples:
Original document accuracy: 0.836 +/- 0.036
Viterbi-corrected accuracy: 0.919 +/- 0.037

For text of length 500, based on 30 samples:
Original document accuracy: 0.834 +/- 0.019
Viterbi-corrected accuracy: 0.908 +/- 0.014

For text of length 1000, based on 30 samples:
Original document accuracy: 0.838 +/- 0.009
Viterbi-corrected accuracy: 0.913 +/- 0.010

For text of length 2000, based on 30 samples:
Original document accuracy: 0.837 +/- 0.006
Viterbi-corrected accuracy: 0.909 +/- 0.007



### I see that the accuracy of the original typo samples was consistent for various lengths.  My Viterbi implementation improved the accuracy for all of the lengths tested, and this improvement did not depend on the length of the text.  The precision of the estimates of the accuracy, however, improved as test length was increased.  For a text size of 2000 characters, the estimates are fairly precise, and there is clear evidence that the Viterbi algorithm is improving the accuracy of the text.  
### I did see that the improvement is sensitive to the pseudocounts, and making these very small causes certain states to have extremely low probabilities.  Tuning the values of the pseudocounts may lead to further improvements.  

# Second Order: 

### Intuitively, the second order HMM should give further improvements because it incorporates more information.  It is reasonable to believe that each hidden state does not depend solely on the state before it, rather it could be modeled to depend on the previous state and the state before that.  That is, there is a structure in words in which the a given pair of letters is not independent on the previous letter, so we should be able to see more improvement with a 2nd order model.

## I chose a different implementation than what I saw people discussing on Piazza.  I started the assignment late due to other assignment, job interviews, etc, so I didn't have a chance to discuss my implementation with anyone, but I think that the following should be valid.  

### Rather than considering pairs of characters explicitly, I created two transition matrices.  
### The first is the same one used above, and represents the probability of transitioning to one hidden state from the previous step.  
### The second is similar, and represents the probability of transitioning to one hidden state from the state two steps previous.  
### Both of these matrices are 27 by 27 because they measure the probability of transitioning from a single character to a single character, and my emission matrix is the original 27 by 27 matrix.  
### Now in each iteration, I consider the transition from the previous step to the current step as well as the transition from 2 steps previos to the current step, and I add both log-probabilities.  
### These two transition probabilities should be weighted, and these weights add a hyperparameter to tune.  Due to time limitations and the extremely long runtime added by the additional 'for' statement to consider the two step-prior transition, I did not experiment with the weights thoroughly, and mostly stuck with 80% importance placed on the one-step-prior transition and 20% importance placed on the two-step-prior transition.  

In [259]:
# Transition matrix for one character to the position two spaces ahead of it
transition2 = np.zeros((27,27))
for i in range(len(s_correct)-2):
    # Get the character pair and the character that follows it
    two_back = s_correct[i]
    next_char = s_correct[i+2]
    
    # Add a count to the transition matrix for each combination
    transition2[char_dict[two_back],char_dict[next_char]] += 1
    
# Normalize the rows
for i in range(transition2.shape[0]):
    row = transition2[i,:]
    transition2[i,:] = row/(np.sum(row))

### With this in place, the modifications to the earlier code are minimal:

In [288]:
def viterbi2(obs, start_pr=start_probs, em=emission, tr=transition, tr2=transition2,\
            e_pseudo = 0.000001, t_pseudo = 0.000001, p_pseudo=0.000001):
    ########### I have added the second transition matrix to the list of arguments
    
    # add pseudocounts to make all elements non-zero
    e = em + e_pseudo
    t = tr + t_pseudo
    t2 = tr + t_pseudo ####### add pseudocounts as to the rest
    start_p = start_pr + p_pseudo
    
    # Array of probabilities for each character at each step in the document
    viterbi_probs = np.zeros((27,len(obs)))

    # Initial state:
    first_char = obs[0]
    # I use log probabilities to avoid underflow
    viterbi_probs[:,0] = np.log(start_p) + np.log(e[char_dict[first_char],:])
    
    ###### Now I also need to initialize the second step.  I do so as I would have performed the first
    ######    iteration of the last implementation.  
    char = obs[1]
    char_idx = char_dict[char]
    for j in range(len(char_dict)):
        prob = max(viterbi_probs[k,0] + np.log(t[k,j]) + np.log(e[j,char_idx]) for k in range(len(char_dict)))
        viterbi_probs[j,1] = prob

    ##### The rest starts at the third step, and each result depends on the previous 2.  
    for i in range(2,len(obs)):
        
        # At each step, find the character and use it to index the appropriate matrix entries
        char = obs[i]
        char_idx = char_dict[char]
        
        for j in range(len(char_dict)):
            # Find the max of the log probabilities for all possible characters
            
            ###### In viterbi 2, I add the log-probability of transitioning to this state from the state
            ######     two steps previous.  This adds another 'for loop' and slows the code considerably.  
            ######
            ###### Here, I weight the importance of each step at 80%/20%.
            ######
            ###### This is my favorite pair of percentages, but it would be great to experiment with 
            ######     different weightings to test the importance of the character two steps prior.  
            ######
            
            prob = max([(0.8 * viterbi_probs[k,i-1] + 0.8 * np.log(t[k,j]) + \
                         0.2 * viterbi_probs[k2,i-2] + 0.2 * np.log(t2[k2,j]) +\
                         + np.log(e[j,char_idx])) for k in range(len(char_dict)) for k2 in range(len(char_dict))])
            viterbi_probs[j,i] = prob
            
    # Predictions are the max probability for each step
    char_predictions = np.argmax(viterbi_probs,axis=0)
    
    # Translate the max probabilities back to characters
    s_viterbi = ''
    for i in char_predictions:
        s_viterbi += index_dict[i]
        
    return s_viterbi

In [289]:
# I create a function to test the algorithm on random slices of a specified length:
######## Modified minimally from above.

def test_viterbi2(length):
    # Random starting position
    start = np.random.randint(low=0, high = len(s_correct)-length)
    
    # Pick out the random segment:
    typo_segment = s_typo[start:start+length]
    correct_segment = s_correct[start:start+length]
    
    # Run viterbi:
    corrected_segment = viterbi2(typo_segment)
    
    # Return accuracy of the original segment and the corrected segment
    return accuracy(typo_segment, correct_segment), accuracy(corrected_segment, correct_segment)

In [290]:
def test_sample2(size,samples):
    print "For text of length {}, based on {} samples:".format(size,samples)
    samples = np.zeros((samples,2))
    for i in range(samples.shape[0]):
        orig, corrected = test_viterbi2(size)  ####### The only change from above
        samples[i,0] = orig
        samples[i,1] = corrected
    print "Original document accuracy: {:.3f} +/- {:.3f}".format(np.mean(samples[:,0]),np.std(samples[:,0]))
    print "Viterbi-corrected accuracy: {:.3f} +/- {:.3f}".format(np.mean(samples[:,1]),np.std(samples[:,1]))

In [291]:
%%time
test_sample2(100,25)

For text of length 100, based on 25 samples:
Original document accuracy: 0.849 +/- 0.029
Viterbi-corrected accuracy: 0.904 +/- 0.030
CPU times: user 3min 18s, sys: 2.01 s, total: 3min 20s
Wall time: 3min 20s


In [292]:
%%time
test_sample2(500,25)

For text of length 500, based on 25 samples:
Original document accuracy: 0.836 +/- 0.014
Viterbi-corrected accuracy: 0.902 +/- 0.015
CPU times: user 17min 28s, sys: 11.1 s, total: 17min 39s
Wall time: 17min 45s


In [293]:
%%time
test_sample2(1000,25)

For text of length 1000, based on 25 samples:
Original document accuracy: 0.838 +/- 0.009
Viterbi-corrected accuracy: 0.900 +/- 0.011
CPU times: user 33min 41s, sys: 21.3 s, total: 34min 3s
Wall time: 34min 4s


In [ ]:
%%time
test_sample2(2000,25)

#### I ran out of time to run all of the tests, but my initial results with the 2nd order HMM were not promising.  The results were comparable to the 1st order model.  
#### With more time, I would experiment with the pseudocounts and the weights of 1-step and 2-step to see if these could improve the accuracy of the corrected text.  


# Extra Problem 2: Final Project Review
    
You will be contacted shortly by a TF to meet and discuss your final project proposal. Be sure to take advantage of this feedback option. Review meetings should be scheduled within the week from April 11-15. 

#### We spoke with Xide on 4/15/16